# UV-Guider alignement with source in autocoll
##  Field 2

In [5]:
%matplotlib inline

from __future__ import division, print_function

import sys
sys.path.insert(1, './Calibration')

import numpy as np
from astropy.io import fits
from astropy import wcs, coordinates
from astropy import units as u
from astropy.wcs.utils import proj_plane_pixel_scales
from matplotlib import pyplot as plt
from astropy.table import Table
import matplotlib.patches as patches

from guider2UV.guider2UV import Guider2UV, diff_skycoord

from Calibration.mapping import Mapping

In [6]:
cloudpath = '/home/dvibert/ownCloud/FIREBALL/'
path_SC_GUI02 = cloudpath + 'Tests-at-FortSumner/170909_SC_GUI02/'

#mask_plate_scale = 42.26134 #mm/deg

#detector platescale
#yscale = 1.106 # arcsec/pix spatial (y det -> y mask)
#xscale = 1.271 # arcsec/pix spectral (x det -> - x mask)

#guider center => put source on guider center
gc = np.array([640, 540])

## calibrated frame encoder gains:

EL : 0.092 x 1.00379 = 0.09469 arcsec/inc

CE : 0.01 x 1.02928 = 0.010293 arcsec/inc

In [7]:
# encoder gains:

CEg = 1.02928
Elg = 1.00379

# 2018 calibrated gains put in CNES software
#CEg = 1.0
#Elg = 1.0

## initialization from last xy calib

In [8]:
G2UV = Guider2UV(filename=path_SC_GUI02 + 'Guider2UV_F2_nogamma.new.pkl')
gc_coord = G2UV.GuiderP.pix2local([gc])

target_filename = '/home/dvibert/ownCloud/FIREBALL/Target_selection/targets_F2.txt'
F2 = Table.read(target_filename, format='ascii')

FOV center angular position in guider <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.39837843, 36.42897653)>): (lon, lat) in deg
    (0.18293516, -0.01420325)>
FOV center pixel position in guider [array(1357.77258408), array(471.30507922)]


## Diffuse UV image

load new mapping

In [9]:
#diffuse_path = '/data/ownCloud/FIREBALL/TestsFTS2018-Flight/AIT-Optical-FTS-2018-Flight/XYCalibration/DiffuseMasksIllumination180823/'
#diffuse_tab_file = diffuse_path + 'StackedImage_24-43-NoDark_table.csv'
#diffuse_tab = Table.read(diffuse_tab_file, format='csv')
mask_det_map_file = '/home/dvibert/work/FireBallPipe/mappings_mask_det_180612/mapping-mask-det-F2.pkl'
mask_det_map = Mapping(mask_det_map_file)
dxm_dyd = -0.013
dym_dxd = -0.015
m = np.array([[0., dxm_dyd],[dym_dxd ,0.]])

record shift from mapping

positive: current diffuse -> mapping (old diffuse)

In [11]:
diffuse_offset = [ 3.7+1, 3.+1] 
direct_map = lambda w, x, y: mask_det_map.map(w, x, y) + diffuse_offset
inv_map = lambda w, x, y: mask_det_map.inv_map(w, x - diffuse_offset[0], y - diffuse_offset[1])

## Scan sequence


gc - 37 - 42 - 38 - 27 - 13 - 9 - 8 - 26- newmodel - s36 - s40 - s46 - s21

### Go from guider to 1st slit: #37

In [12]:
slit_pos1 =   np.array([F2[F2['Internal-count']=='37']['xmm'][0], F2[F2['Internal-count']=='37']['ymm'][0]])

print("slit position in mm on mask:", slit_pos1)
slit_coord1 = G2UV.SienceMask2guider(slit_pos1, angle=True)
print("slit coord in guider local frame:", slit_coord1.to_string(u'dms'))

#frame move
theta_EL =  slit_coord1.lon.deg/2./Elg
theta_CE = -slit_coord1.lat.deg/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

slit position in mm on mask: [ 4.02636 -5.77259]
slit coord in guider local frame: ['0d02m59.2244s -0d06m52.663s']
EL: [89.27384235] arcsec 
CE: [200.46196137] arcsec


### Go from guider to 1st slit: #37

remove offset from model to prediction

In [13]:
slit_coord1_det = direct_map(mask_det_map.w[1], slit_pos1[0], slit_pos1[1])
print("mapped slit position on detector: ", slit_coord1_det)

mapped slit position on detector:  [1912.32192028  544.738795  ]


In [14]:
# compute observed position on mask
slit_coord1_obs_det = np.array([1908.3, 565.0]) # observed
slit_coord1_obs_wave = 0.20619
slit_pos1_obs = inv_map(slit_coord1_obs_wave, slit_coord1_obs_det[0], slit_coord1_obs_det[1])
slit_pos1_obs2 = slit_pos1 + m.dot(slit_coord1_obs_det - slit_coord1_det)
print("observed slit position in mm on mask:", slit_pos1_obs)
print("observed slit position (linear mapping) in mm on mask:", slit_pos1_obs2)

slit_coord1_obs = G2UV.SienceMask2guider(slit_pos1_obs, angle=True)
print("observed slit coord in guider local frame:", slit_coord1_obs.to_string(u'dms'))
slit_coord1_obs_pix = G2UV.SienceMask2guider(slit_pos1_obs, angle=False)
print("observed slit coord in guider pixel frame:", slit_coord1_obs_pix)


observed slit position in mm on mask: [ 3.75248214 -5.72902049]
observed slit position (linear mapping) in mm on mask: [ 3.76296433 -5.7122612 ]
observed slit coord in guider local frame: ['0d03m02.0895s -0d06m29.1641s']
observed slit coord in guider pixel frame: [array([850.73712814]), array([97.76556668])]


In [15]:
offset =  diff_skycoord(slit_coord1_obs, slit_coord1) 
print("offset from guider in arcsec (guider frame): \n",offset*3600)


offset from guider in arcsec (guider frame): 
 [[ -2.86511642]
 [-23.49890734]]


In [16]:
G2UVnew  = G2UV.copy()

newFOVcenter = coordinates.SkyCoord(G2UV.FOV_center_guider_coord.lon + offset[0]*u.deg, 
                              G2UV.FOV_center_guider_coord.lat + offset[1]*u.deg,
                              frame = G2UV.GuiderP.localframe)
                     
G2UVnew.FOV_center_guider_coord = newFOVcenter

In [17]:
slit_coord1_cor = G2UVnew.SienceMask2guider(slit_pos1, angle=True)
print("slit coord in guider local frame, corrected from offset:", slit_coord1_cor.to_string(u'dms'))


slit coord in guider local frame, corrected from offset: ['0d02m56.3593s -0d07m16.1619s']


### go back to guider center (absolute)

go to 1st slit, offset removed:

In [18]:

slit_coord1 = G2UVnew.SienceMask2guider(slit_pos1, angle=True)
print("slit coord in guider local frame:", slit_coord1.to_string(u'dms'))

#frame move
theta_EL =  slit_coord1.lon.deg/2./Elg
theta_CE = -slit_coord1.lat.deg/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

slit coord in guider local frame: ['0d02m56.3593s -0d07m16.1619s']
EL: [87.84669303] arcsec 
CE: [211.87717751] arcsec


### Loop on slits

In [19]:
slits = ['37', '42', '38', '27', '13', '9', '8', '26'] 
slits += slits[::-1]
slits

['37',
 '42',
 '38',
 '27',
 '13',
 '9',
 '8',
 '26',
 '26',
 '8',
 '9',
 '13',
 '27',
 '38',
 '42',
 '37']

In [21]:
slit_pos = np.zeros((len(slits),2))
slit_coord = []
slit_coord_pix = np.zeros((len(slits),2))
theta_EL = np.zeros(len(slits))
theta_CE = np.zeros(len(slits))

for i, s in enumerate(slits):
    
    slit_pos[i] = np.array([F2[F2['Internal-count']==s]['xmm'][0], F2[F2['Internal-count']==s]['ymm'][0]])
    print("slit position in mm on mask:", slit_pos[i])
    slit_coord.append(G2UVnew.SienceMask2guider(slit_pos[i], angle=True))
    print("slit coord in guider local frame:", slit_coord[-1].to_string(u'dms'))
    slit_coord_pix[i] = np.array(G2UVnew.SienceMask2guider(slit_pos[i], angle=False)).T
    print("slit coord in guider pixel frame:", slit_coord_pix[i])

    #frame move
    theta_EL[i] =  slit_coord[-1].lon.deg/2./Elg
    theta_CE[i] = -slit_coord[-1].lat.deg/2./CEg
    print("EL: {:.1f} arcsec \nCE: {:.1f} arcsec".format(theta_EL[i]*3600, theta_CE[i]*3600))
    
# compute relative moves
moves_EL = theta_EL[1:] - theta_EL[:-1]
moves_CE = theta_CE[1:] - theta_CE[:-1]


slit position in mm on mask: [ 4.02636 -5.77259]
slit coord in guider local frame: ['0d02m56.3593s -0d07m16.1619s']
slit coord in guider pixel frame: [846.06404759  43.78582257]
EL: 87.8 arcsec 
CE: 211.9 arcsec
slit position in mm on mask: [ 6.431447 -0.117245]
slit coord in guider local frame: ['0d11m06.0857s -0d10m23.2024s']
slit coord in guider pixel frame: [1386.77126104 -170.61650828]
EL: 331.8 arcsec 
CE: 302.7 arcsec
slit position in mm on mask: [4.334963 3.091563]
slit coord in guider local frame: ['0d15m32.8996s -0d07m14.2661s']
slit coord in guider pixel frame: [1655.4229569    36.78199493]
EL: 464.7 arcsec 
CE: 211.0 arcsec
slit position in mm on mask: [-0.55449  -2.841184]
slit coord in guider local frame: ['0d06m51.8901s -0d00m36.3885s']
slit coord in guider pixel frame: [1094.7319317   494.52746691]
EL: 205.2 arcsec 
CE: 17.7 arcsec
slit position in mm on mask: [-6.6486    4.448872]
slit coord in guider local frame: ['0d16m53.9676s 0d08m26.3723s']
slit coord in guider pi

In [22]:
# remove middle
idx = np.arange(len(slits)//2)
idx = np.concatenate((idx[:-1], idx[:-1]+len(idx)))
for i in idx:
    print("move:  EL {:.1f} arcsec - CE {:.1f} arcsec".format(moves_EL[i]*3600, moves_CE[i]*3600))


move:  EL 243.9 arcsec - CE 90.9 arcsec
move:  EL 132.9 arcsec - CE -91.8 arcsec
move:  EL -259.5 arcsec - CE -193.3 arcsec
move:  EL 299.9 arcsec - CE -263.7 arcsec
move:  EL -172.9 arcsec - CE -67.2 arcsec
move:  EL -166.5 arcsec - CE -2.8 arcsec
move:  EL -92.6 arcsec - CE 273.9 arcsec
move:  EL 92.6 arcsec - CE -273.9 arcsec
move:  EL 166.5 arcsec - CE 2.8 arcsec
move:  EL 172.9 arcsec - CE 67.2 arcsec
move:  EL -299.9 arcsec - CE 263.7 arcsec
move:  EL 259.5 arcsec - CE 193.3 arcsec
move:  EL -132.9 arcsec - CE 91.8 arcsec
move:  EL -243.9 arcsec - CE -90.9 arcsec


observed slits position :

In [ ]:
#slit_coord_obs_det = np.zeros((8,2)) + np.array([1800, 421])

#slit_coord_obs_det = np.array([[ 1803 ,  421.2 ],
#         [ 1469.6 ,  288.7 ],
#         [ 1304.2 , 1347.9 ],
#         [ 1619.6 ,  1625.4 ],
#         [ 1620 ,  1626.6 ],
#
#         [ 1304.3 ,  1355.6 ],
#         [ 1469.5 ,  288.7 ],
#         [ 1802.9 ,  412.5 ]])
slit_coord_obs_det

In [ ]:
mask_det_map.map(mask_det_map.w[1], slit_pos[:,0], slit_pos[:, 1]).T + diffuse_offset
#direct_map(mask_det_map.w[1], slit_pos[:,0].T, slit_pos[:, 1].T)

map on mask :

In [ ]:
slit_coord_obs_wave = 0.20619    

#slit_pos_obs = mask_det_map.inv_map(slit_coord_obs_wave, slit_coord_obs_det[:,0], slit_coord_obs_det[:,1]).T
slit_pos_obs = inv_map(slit_coord_obs_wave, slit_coord_obs_det[:,0], slit_coord_obs_det[:,1]).T
#print(slit_pos_obs - slit_pos)

#print(slit_coord_obs_wave)
#print(slit_pos_obs)


In [ ]:
slit_coord_obs = []
slit_coord_obs_pix = np.zeros((len(slits),2))

for i, s in enumerate(slits):
    
    print("observed slit position in mm on mask:", slit_pos_obs[i])
    slit_coord_obs.append(G2UVnew.SienceMask2guider(slit_pos_obs[i], angle=True))
    print("observed slit coord in guider local frame:", slit_coord_obs[-1].to_string(u'dms'))
    slit_coord_obs_pix[i] = np.array(G2UVnew.SienceMask2guider(slit_pos_obs[i], angle=False)).T
    print("observed slit coord in guider pixel frame:", slit_coord_obs_pix[i])



## compute NEW MODEL: orientation, offset & magnification

In [ ]:
slit_coord_F = np.zeros((len(slits),2))
slit_coord_obs_F = np.zeros((len(slits),2))

for i in range(len(slits)):
    
    c = G2UVnew.guider_to_FieldLocal(slit_coord[i], angle=True)
    slit_coord_F[i] = np.array([ c.lon.deg, c.lat.deg]).T
    
    c_obs = G2UVnew.guider_to_FieldLocal(slit_coord_obs[i], angle=True)
    slit_coord_obs_F[i] = np.array([ c_obs.lon.deg, c_obs.lat.deg]).T

    
delta = slit_coord_obs_F - slit_coord_F
print(delta*3600)


In [ ]:
row_x = np.hstack((slit_coord_F*[1.,-1], np.ones((8,1)), np.zeros((8,1)) )) # xn -x = x dgamma - y theta + dx
row_y = np.hstack((slit_coord_F[:,::-1], np.zeros((8,1)), np.ones((8,1)) )) # yn - y = x theta - y dgamma 
mat = np.vstack((row_x, row_y))
matinv =  np.linalg.pinv(mat)
data = np.concatenate((delta[:,0], delta[:,1]))
sol = matinv.dot(data)
gama = 1 + sol[0]
theta_rad = sol[1]
deltax = sol[2]
deltay = sol[3]
theta = theta_rad*180/np.pi*60 #arcmin
print("gamma: {}\ntheta: {} arcmin\ndx: {} arcsec\ndy: {} arcsec".format(gama, theta, deltax*3600, deltay*3600))
covar = matinv.dot(matinv.T)
# accuracy, assuming 1 arcsec measurement error
print("variances: {}\n".format(np.sqrt(np.diag(covar))/3600*[1, 180/np.pi*60, 3600, 3600])) #
#residual
data_new = mat.dot(sol) 
print("residuals in arcsec:", (data_new - data)*3600)

In [ ]:
mask_plate_scale = 42.26134
plt.figure()
plt.axis('equal')
plt.plot(slit_pos[:,0], slit_pos[:,1], 'or')
qv = plt.quiver(slit_pos[:,0], slit_pos[:,1], 0., delta[:,0]*mask_plate_scale)
plt.quiverkey(qv, .8,.9,.075, "75 mu", color='r')
plt.xlim([-13, 13])
plt.ylim([-7,7])
plt.xlabel('x mm')
plt.ylabel('y mm')
plt.text(-10,7,"rotation: {:.2f} arcmin\nmagnification {:.4f}\ndeltax: {:.4f} arcsec\ndeltay: {:.4f} arcsec".format(theta, gama, -deltay*3600, deltax*3600))



In [ ]:
sit_pos14_obs_det = np.array([1312.8, 1357.7])
slit_pos14_obs = inv_map(slit_coord_obs_wave, sit_pos14_obs_det[0], sit_pos14_obs_det[1]).T
print(slit_pos14_obs, slit_pos[2])

print(G2UV.FieldP.pix2local(slit_pos[2]))
print(G2UV.FieldP.pix2local(slit_pos14_obs))

In [ ]:
G2UVcor  = Guider2UV(guider_wcs=G2UVnew.GuiderP.w, 
                     Field_center=G2UVnew.FieldP.center, 
                     Field_rotation=G2UVnew.FieldP.rotation,
                     Field_gamma=gama,
                     mask_rotation=G2UVnew.mask_rotation + theta*u.arcmin)


#deltaFOV = - 1/gama*deltax

#G2UVnew  = Guider2UV(guider_wcs=G2UV.GuiderP.w, 
#                     Field_center=G2UV.FieldP.center, 
#                     Field_rotation=G2UV.FieldP.rotation,
#                     Field_gamma=gama)
#                     #mask_rotation=theta*u.arcmin)
#
#deltaFOV = - deltax

newFOVcenter = coordinates.SkyCoord(G2UVnew.FOV_center_guider_coord.lon + deltax*u.deg, 
                              G2UVnew.FOV_center_guider_coord.lat + deltay*u.deg,
                              frame = G2UVnew.GuiderP.localframe)
                     
G2UVcor.FOV_center_guider_coord = newFOVcenter

#G2UVnew.FOV_guider_coord = G2UV.FOV_guider_coord


## check new model on new slit #??

start from last slit #26

In [ ]:
slitcheck = '26'
slit_pos5 =   np.array([F2[F2['Internal-count']==slitcheck]['xmm'][0], F2[F2['Internal-count']==slitcheck]['ymm'][0]])

print("slit position in mm on mask:", slit_pos5)
slit_coord5 = G2UVcor.SienceMask2guider(slit_pos5, angle=True)
print("slit coord in guider local frame:", slit_coord5.to_string(u'dms'))
slit_coord5_pix = G2UVcor.SienceMask2guider(slit_pos5, angle=False)
print("slit coord in guider pixel frame:", slit_coord5_pix)

#frame move
theta_EL =  (slit_coord5.lon.deg -  slit_coord[0].lon.deg)/2./Elg
theta_CE = -(slit_coord5.lat.deg -  slit_coord[0].lat.deg)/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

## check new model on known slit #??

start from last slit #14

In [ ]:
slits

In [ ]:
slitcheck2 = slits[2]
slit_pos6 =   np.array([F2[F2['Internal-count']==slitcheck2]['xmm'][0], F2[F2['Internal-count']==slitcheck2]['ymm'][0]])

print("slit position in mm on mask:", slit_pos6)
slit_coord6 = G2UVcor.SienceMask2guider(slit_pos6, angle=True)
print("slit coord in guider local frame:", slit_coord6.to_string(u'dms'))
slit_coord6_pix = G2UVcor.SienceMask2guider(slit_pos6, angle=False)
print("slit coord in guider pixel frame:", slit_coord6_pix)

#frame move
theta_EL =  (slit_coord6.lon.deg -  slit_coord5.lon.deg)/2./Elg
theta_CE = -(slit_coord6.lat.deg -  slit_coord5.lat.deg)/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

compared observed vs diffuse

| wavelength | diffuse   | observed |
|------------|-----------|----------|
| 0.21382    |           |           
| 0.20619    | 1738.31531.2| 1754.3x1574.3
| 0.20255    |   |


In [ ]:
# compute observed position on mask
slit_coord5_obs_det = np.array([1754.3, 1574.3])
slit_coord5_obs_wave = 0.20619
slit_pos5_obs = mask_det_map.inv_map(slit_coord5_obs_wave, slit_coord5_obs_det[0], slit_coord5_obs_det[1])
print("observed slit position in mm on mask:", slit_pos5_obs)
slit_coord5_obs = G2UVnew.SienceMask2guider(slit_pos5_obs, angle=True)
print("observed slit coord in guider local frame:", slit_coord5_obs.to_string(u'dms'))
slit_coord5_obs_pix = G2UVnew.SienceMask2guider(slit_pos5_obs, angle=False)
print("observed slit coord in guider pixel frame:", slit_coord5_obs_pix)


## F2 Star sequence

gc - s36 - s40 - s46 - s21


In [24]:
star_target_path = cloudpath + 'Target_selection/GuidingStars/'
F2_stars = Table.read(star_target_path + "F2_guidingstars.fits", format='fits')
F2_stars_coords = coordinates.SkyCoord(F2_stars['RA']*u.deg, F2_stars['DEC']*u.deg)


### star s36

In [25]:
star_pos1_radec = F2_stars_coords[F2_stars['Internal count']==36]
print("star position Ra/Dec: ", star_pos1_radec)
star_coord1 = G2UVnew.SienceMask2guider(star_pos1_radec, world=True, angle=True)
print("Star position in guider local frame: ", star_coord1)
star_coord1_pix = G2UVnew.SienceMask2guider(star_pos1_radec, world=True, angle=False)
print("Star position in guider pixel frame: ", star_coord1_pix)

#frame move
theta_EL =  star_coord1.lon.deg/2./Elg
theta_CE = -star_coord1.lat.deg/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

star position Ra/Dec:  <SkyCoord (ICRS): (ra, dec) in deg
    [(252.895501, 34.875333)]>
Star position in guider local frame:  <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.39837843, 36.42897653)>): (lon, lat) in deg
    [(0.02272687, -0.06864028)]>
Star position in guider pixel frame:  [array([734.74174777]), array([259.57792384])]
EL: [40.75390833] arcsec 
CE: [120.03780446] arcsec


obs star: 736.5x271.2 736.3x264.0

### star s40

In [9]:
star_pos2_radec = F2_stars_coords[F2_stars['Internal count']==40]
print("star position Ra/Dec: ", star_pos2_radec)
star_coord2 = G2UVnew.SienceMask2guider(star_pos2_radec, world=True, angle=True)
print("Star position in guider local frame: ", star_coord2)
star_coord2_pix = G2UVew.SienceMask2guider(star_pos2_radec, world=True, angle=False)
print("Star position in guider pixel frame: ", star_coord2_pix)

#frame move
#theta_EL =  star_coord2.lon.deg/2./Elg
#theta_CE = -star_coord2.lat.deg/2./CEg
#print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

theta_EL =  (star_coord2.lon.deg - star_coord1.lon.deg)/2./Elg
theta_CE = -(star_coord2.lat.deg - star_coord1.lat.deg)/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

star position Ra/Dec:  <SkyCoord (ICRS): (ra, dec) in deg
    [(252.83754, 34.836825)]>
Star position in guider local frame:  <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.39837843, 36.42897653)>): (lon, lat) in deg
    [(-0.03364437, -0.08407031)]>
Star position in guider pixel frame:  [array([505.29012519]), array([192.79828529])]
EL: [-102.51225968] arcsec 
CE: [38.39916836] arcsec


obs star: 505.2x176.9 retour 504.8x171.4

### star s46

In [10]:
star_pos3_radec = F2_stars_coords[F2_stars['Internal count']==46]
print("star position Ra/Dec: ", star_pos3_radec)
star_coord3 = G2UVnew.SienceMask2guider(star_pos3_radec, world=True, angle=True)
print("Star position in guider local frame: ", star_coord3)
star_coord3_pix = G2UVnew.SienceMask2guider(star_pos3_radec, world=True, angle=False)
print("Star position in guider pixel frame: ", star_coord3_pix)

#frame move
#theta_EL =  star_coord2.lon.deg/2./Elg
#theta_CE = -star_coord2.lat.deg/2./CEg
#print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

theta_EL =  (star_coord3.lon.deg - star_coord2.lon.deg)/2./Elg
theta_CE = -(star_coord3.lat.deg - star_coord2.lat.deg)/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

star position Ra/Dec:  <SkyCoord (ICRS): (ra, dec) in deg
    [(252.787662, 34.810634)]>
Star position in guider local frame:  <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.39837843, 36.42897653)>): (lon, lat) in deg
    [(-0.08071486, -0.09632337)]>
Star position in guider pixel frame:  [array([309.19014692]), array([136.41712945])]
EL: [-84.40698687] arcsec 
CE: [21.42809089] arcsec


obs star 307.7x122.7 retour: 307.8x122.5

### star s21

In [11]:
star_pos4_radec = F2_stars_coords[F2_stars['Internal count']==21]
print("star position Ra/Dec: ", star_pos4_radec)
star_coord4 = G2UVnew.SienceMask2guider(star_pos4_radec, world=True, angle=True)
print("Star position in guider local frame: ", star_coord4)
star_coord4_pix = G2UVnew.SienceMask2guider(star_pos4_radec, world=True, angle=False)
print("Star position in guider pixel frame: ", star_coord4_pix)

#frame move
#theta_EL =  star_coord2.lon.deg/2./Elg
#theta_CE = -star_coord2.lat.deg/2./CEg
#print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

theta_EL =  (star_coord4.lon.deg - star_coord3.lon.deg)/2./Elg
theta_CE = -(star_coord4.lat.deg - star_coord3.lat.deg)/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

star position Ra/Dec:  <SkyCoord (ICRS): (ra, dec) in deg
    [(252.829951, 34.945769)]>
Star position in guider local frame:  <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (250.39837843, 36.42897653)>): (lon, lat) in deg
    [(-0.0058759, 0.02153703)]>
Star position in guider pixel frame:  [array([616.35244458]), array([627.49245875])]
EL: [134.20150808] arcsec 
CE: [-206.1136998] arcsec


obs star 611.9x600 - retour: 611.5x605.7

### back to guider center

In [12]:
#frame move
theta_EL =  (0. - star_coord4.lon.deg)/2./Elg
theta_CE = -(0. - star_coord4.lat.deg)/2./CEg
print("EL: {} arcsec \nCE: {} arcsec".format(theta_EL*3600, theta_CE*3600))

EL: [10.53668082] arcsec 
CE: [37.66385224] arcsec
